In [8]:
from abc import ABC, abstractmethod
from typing import List, Union

# Visitor Interface
class Visitor(ABC):
    @abstractmethod
    def visit_file(self, file: 'File') -> None:
        pass

    @abstractmethod
    def visit_directory(self, directory: 'Directory') -> None:
        pass

# Element Interface
class Element(ABC):
    @abstractmethod
    def accept(self, visitor: Visitor) -> None:
        pass

# Concrete Element: File
class File(Element):
    def __init__(self, name: str, size: int) -> None:
        self.name: str = name
        self.size: int = size  # Size in bytes

    def accept(self, visitor: Visitor) -> None:
        visitor.visit_file(self)

# Concrete Element: Directory
class Directory(Element):
    def __init__(self, name: str) -> None:
        self.name: str = name
        self.children: List[Element] = []  # Can contain Files or Directories

    def add(self, element: Element) -> None:
        self.children.append(element)

    def accept(self, visitor: Visitor) -> None:
        visitor.visit_directory(self)

# Concrete Visitor: SizeCalculator
class SizeCalculator(Visitor):
    def __init__(self) -> None:
        self.total_size: int = 0

    def visit_file(self, file: File) -> None:
        print(f"Calculating size of file: {file.name} ({file.size} bytes)")
        self.total_size += file.size

    def visit_directory(self, directory: Directory) -> None:
        print(f"Entering directory: {directory.name}")
        for child in directory.children:
            child.accept(self)
        print(f"Exiting directory: {directory.name}")

# Concrete Visitor: DisplayVisitor
class DisplayVisitor(Visitor):
    def __init__(self) -> None:
        self.indent_level: int = 0

    def visit_file(self, file: File) -> None:
        print('    ' * self.indent_level + f"- File: {file.name} ({file.size} bytes)")

    def visit_directory(self, directory: Directory) -> None:
        print('    ' * self.indent_level + f"+ Directory: {directory.name}")
        self.indent_level += 1
        for child in directory.children:
            child.accept(self)
        self.indent_level -= 1



In [18]:
import random

class ElementFactory:
    def get_random_element(self):
        # Randomly choose to create a File or a Directory
        if random.choice([True, False]):
            # Create a File with a random size
            return File(f"file{random.randint(1, 10)}.txt", random.randint(100, 10000))
        else:
            # Create a Directory and optionally add some Files to it
            directory = Directory(f"directory{random.randint(1, 10)}")
            # Randomly decide to add some files to the directory
            for _ in range(random.randint(0, 3)):  # Add up to 3 files
                directory.add(File(f"nested{random.randint(1, 5)}.txt", random.randint(100, 5000)))
            return directory
        
def main():
    factory = ElementFactory()
    elements = [factory.get_random_element() for _ in range(5)]

    size_calculator = SizeCalculator()
    display_visitor = DisplayVisitor()
    print(f"\nVisualizing all elements...\n")

    # Applying visitors to the elements created by the factory
    for element in elements:
        element.accept(display_visitor)

    print(f"\nCalculating total size of all elements...\n")


    for element in elements:
        element.accept(size_calculator)

    print(f"\nTotal size of all elements: {size_calculator.total_size} bytes")

if __name__ == "__main__":
    main()




Visualizing all elements...

+ Directory: directory5
    - File: nested3.txt (1863 bytes)
    - File: nested4.txt (4394 bytes)
    - File: nested2.txt (2780 bytes)
+ Directory: directory9
    - File: nested2.txt (348 bytes)
    - File: nested2.txt (3996 bytes)
    - File: nested3.txt (272 bytes)
+ Directory: directory6
    - File: nested2.txt (3526 bytes)
    - File: nested1.txt (4778 bytes)
+ Directory: directory2
    - File: nested3.txt (4944 bytes)
- File: file10.txt (9118 bytes)

Calculating total size of all elements...

Entering directory: directory5
Calculating size of file: nested3.txt (1863 bytes)
Calculating size of file: nested4.txt (4394 bytes)
Calculating size of file: nested2.txt (2780 bytes)
Exiting directory: directory5
Entering directory: directory9
Calculating size of file: nested2.txt (348 bytes)
Calculating size of file: nested2.txt (3996 bytes)
Calculating size of file: nested3.txt (272 bytes)
Exiting directory: directory9
Entering directory: directory6
Calculatin

In [10]:
from abc import ABC, abstractmethod

# Visitor Interface
class ExpressionVisitor(ABC):
    @abstractmethod
    def visit_number(self, number):
        pass

    @abstractmethod
    def visit_addition(self, addition):
        pass

    @abstractmethod
    def visit_multiplication(self, multiplication):
        pass

# Element Interface
class Expression(ABC):
    @abstractmethod
    def accept(self, visitor):
        pass

# Concrete Element: Number
class Number(Expression):
    def __init__(self, value):
        self.value = value

    def accept(self, visitor):
        visitor.visit_number(self)

# Concrete Element: Addition
class Addition(Expression):
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def accept(self, visitor):
        visitor.visit_addition(self)

# Concrete Element: Multiplication
class Multiplication(Expression):
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def accept(self, visitor):
        visitor.visit_multiplication(self)

# Concrete Visitor: EvaluationVisitor
class EvaluationVisitor(ExpressionVisitor):
    def __init__(self):
        self.stack = []

    def visit_number(self, number):
        print(f"Visiting Number: {number.value}")
        self.stack.append(number.value)

    def visit_addition(self, addition):
        print("Visiting Addition")
        addition.left.accept(self)
        addition.right.accept(self)
        right = self.stack.pop()
        left = self.stack.pop()
        result = left + right
        print(f"Adding {left} + {right} = {result}")
        self.stack.append(result)

    def visit_multiplication(self, multiplication):
        print("Visiting Multiplication")
        multiplication.left.accept(self)
        multiplication.right.accept(self)
        right = self.stack.pop()
        left = self.stack.pop()
        result = left * right
        print(f"Multiplying {left} * {right} = {result}")
        self.stack.append(result)

In [11]:
# Client Code
def main():
    # Represent the expression: (5 + 3) * 2
    expr = Multiplication(
        Addition(Number(5), Number(3)),
        Number(2)
    )

    evaluator = EvaluationVisitor()
    expr.accept(evaluator)
    result = evaluator.stack.pop()
    print(f"\nResult of the expression: {result}")

if __name__ == "__main__":
    main()


Visiting Multiplication
Visiting Addition
Visiting Number: 5
Visiting Number: 3
Adding 5 + 3 = 8
Visiting Number: 2
Multiplying 8 * 2 = 16

Result of the expression: 16
